In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Change working directory to root of project
import os
os.chdir('..')

In [148]:
import os
from datetime import datetime
import glob

from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k
import pandas as pd
import praw
import numpy as np
from dateutil.relativedelta import relativedelta
from pprint import pprint
from IPython.display import Image

from beam_recommender.recommender import Recommender
import beam_recommender.helpers as h
import beam_recommender.config as config
from beam_recommender.data import BeamInteractions, RedditInteractions

In [28]:
r = praw.Reddit(client_id=config.REDDIT_CLIENT_ID,
                client_secret=config.REDDIT_CLIENT_SECRET,
                user_agent=config.REDDIT_USER_AGENT)

In [45]:
reddit_interactions_df = RedditInteractions.get_interactions()

In [46]:
beam_interactions_df = BeamInteractions.get_interactions_with_weights()

In [51]:
del reddit_interactions_df['subreddit']

In [52]:
reddit_interactions_df['weight'] = 1

In [56]:
del beam_interactions_df['last_interaction']

In [57]:
interactions_df = reddit_interactions_df.append(beam_interactions_df, ignore_index=True)

In [65]:
interactions_filtered_df = h.threshold_interactions_df(interactions_df, 'user', 'subreddit_id', 10, 10)

Starting interactions info
Number of rows: 4399754
Number of cols: 83824
Sparsity: 0.006%
Ending interactions info
Number of rows: 524660
Number of columns: 21041
Sparsity: 0.101%


In [66]:
recommender = Recommender(interactions_filtered_df, dict(num_threads=4))

In [31]:
recommender.train()

In [67]:
recommender.evaluate()

Precision@10 0.121237
Recall@10 0.242473220752
AUC 0.949625
Reciprocal Rank 0.430833


In [160]:
recommender.recommend(['timonvs'], 40)

[['learnprogramming',
  'programming',
  'GifRecipes',
  'Jokes',
  'pcmasterrace',
  'cscareerquestions',
  'UpliftingNews',
  'nottheonion',
  'AskDocs',
  'The_Donald',
  'pokemon',
  'Surface',
  'Advice',
  'olympics',
  'FoodPorn',
  'personalfinance',
  'Baking',
  'cringepics',
  'ideasfortheadmins',
  'InternetIsBeautiful',
  'NintendoSwitch',
  'gamedev',
  'Android',
  'grilledcheese',
  'space',
  'gifsthatendtoosoon',
  'TumblrInAction',
  'learnjavascript',
  'france',
  'Cooking',
  'linux',
  'interestingasfuck',
  'django',
  'iphone',
  'mildlyinfuriating',
  'PS4',
  'ForeverAlone',
  'teenagers',
  'pussypassdenied',
  'pokemongo']]

In [159]:
# list(r.info(beam_interactions_df[beam_interactions_df['user'] == 'timonvs'].query('last_interaction > @SINCE_LAST_WEEK')['subreddit_id'].tolist()))

In [158]:
# ratings_df.apply(lambda x: 2 if (x['last_interaction'] > SINCE_LAST_WEEK and x['count'] > 4) else 1, axis=1)

In [174]:
subreddit = r.subreddit('japanpics')
for submission in subreddit.hot(limit=20):
    if submission.stickied:
        continue
        
    print('Title:', submission.title)
    print()
    print('Author:', submission.author)
    print()
    print(submission.selftext_html)
    print()
    
    props = vars(submission)
    preview = props.get('preview')
    
    if preview:
        print(preview['images'][-1]['resolutions'][-1]['url'])
    
    print()
    print('https://reddit.com{}'.format(submission.permalink))
    print()
    
    print('---')
    print()
    
#     print(submission.preview)
    
#     if submission.preview. and submission.preview['images']:
#         print(submission.preview['images'][-1])
    
#     pprint(vars(submission))

Title: Tokyo cinematics [OC]

Author: Iamthetophergopher

None

https://i.redditmedia.com/OFF6LTpP6JxZ4rkuveVt-o7h88O5zjX9ZczWu7NHZ0k.jpg?fit=crop&crop=faces%2Centropy&arh=2&w=1080&s=904e9b285d0dbe1af5ac42305d5c7178

https://reddit.com/r/japanpics/comments/5pu7mj/tokyo_cinematics_oc/

---

Title: Reflecting after the deluge, Osaka [OC]

Author: Iamthetophergopher

None

https://i.redditmedia.com/KWX6F2k8NIr-1owa8upckH5olgGFN0vwNdTlyHtSpXM.jpg?fit=crop&crop=faces%2Centropy&arh=2&w=1080&s=865e19f5e6f43684bcf61620d1bb8f10

https://reddit.com/r/japanpics/comments/5pu6ip/reflecting_after_the_deluge_osaka_oc/

---

Title: Decorative wood carving of dragon on a door in Asakusa. (2520x1873)

Author: profbecker

None

https://i.redditmedia.com/7K6Qo1dFDsjsvUe7zVUDANPtjfDY0kkUPBmUuKe1iKo.jpg?fit=crop&crop=faces%2Centropy&arh=2&w=1080&s=6ec828f29a09859e3278450a1ac1a771

https://reddit.com/r/japanpics/comments/5ptwe7/decorative_wood_carving_of_dragon_on_a_door_in/

---

Title: [OC] Shinjuku Reflec

In [175]:
subreddit.public_description

'A subreddit for photos of beautiful Japan!\n\nMember of the /r/NationalPhotoSubs network.'